# Co-Win Web API's

In [1]:
import requests

## 1. Find the ID of our district

In [2]:
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.93 Safari/537.36"}

In [3]:
response = requests.get("https://cdn-api.co-vin.in/api/v2/admin/location/states", headers=headers)
print(response.status_code)

200


In [4]:
response.json()

{'states': [{'state_id': 1, 'state_name': 'Andaman and Nicobar Islands'},
  {'state_id': 2, 'state_name': 'Andhra Pradesh'},
  {'state_id': 3, 'state_name': 'Arunachal Pradesh'},
  {'state_id': 4, 'state_name': 'Assam'},
  {'state_id': 5, 'state_name': 'Bihar'},
  {'state_id': 6, 'state_name': 'Chandigarh'},
  {'state_id': 7, 'state_name': 'Chhattisgarh'},
  {'state_id': 8, 'state_name': 'Dadra and Nagar Haveli'},
  {'state_id': 37, 'state_name': 'Daman and Diu'},
  {'state_id': 9, 'state_name': 'Delhi'},
  {'state_id': 10, 'state_name': 'Goa'},
  {'state_id': 11, 'state_name': 'Gujarat'},
  {'state_id': 12, 'state_name': 'Haryana'},
  {'state_id': 13, 'state_name': 'Himachal Pradesh'},
  {'state_id': 14, 'state_name': 'Jammu and Kashmir'},
  {'state_id': 15, 'state_name': 'Jharkhand'},
  {'state_id': 16, 'state_name': 'Karnataka'},
  {'state_id': 17, 'state_name': 'Kerala'},
  {'state_id': 18, 'state_name': 'Ladakh'},
  {'state_id': 19, 'state_name': 'Lakshadweep'},
  {'state_id': 20,

Say we want the districts in `tamil nadu`, we will get the state's code from the list above. Its value is `31`

In [5]:
# State code for tamil nadu is 31

# Now we get the districts here
response = requests.get("https://cdn-api.co-vin.in/api/v2/admin/location/districts/31", headers=headers)
print(response.status_code)

200


In [6]:
response.json()

{'districts': [{'district_id': 779, 'district_name': 'Aranthangi'},
  {'district_id': 555, 'district_name': 'Ariyalur'},
  {'district_id': 578, 'district_name': 'Attur'},
  {'district_id': 565, 'district_name': 'Chengalpet'},
  {'district_id': 571, 'district_name': 'Chennai'},
  {'district_id': 778, 'district_name': 'Cheyyar'},
  {'district_id': 539, 'district_name': 'Coimbatore'},
  {'district_id': 547, 'district_name': 'Cuddalore'},
  {'district_id': 566, 'district_name': 'Dharmapuri'},
  {'district_id': 556, 'district_name': 'Dindigul'},
  {'district_id': 563, 'district_name': 'Erode'},
  {'district_id': 552, 'district_name': 'Kallakurichi'},
  {'district_id': 557, 'district_name': 'Kanchipuram'},
  {'district_id': 544, 'district_name': 'Kanyakumari'},
  {'district_id': 559, 'district_name': 'Karur'},
  {'district_id': 780, 'district_name': 'Kovilpatti'},
  {'district_id': 562, 'district_name': 'Krishnagiri'},
  {'district_id': 540, 'district_name': 'Madurai'},
  {'district_id': 576

We note down the code for a district, say the code for `madurai` is `540`

## 2. Decide the date to be queried

In [7]:
import datetime
import pytz

In [8]:
# Get the current time
now = datetime.datetime.now()

# Set the timezone
tz = pytz.timezone('Asia/Kolkata')

# Convert the current time into this timezone
now = now.astimezone(tz)

now

datetime.datetime(2021, 8, 11, 12, 2, 52, 82588, tzinfo=<DstTzInfo 'Asia/Kolkata' IST+5:30:00 STD>)

In [9]:
# Get this into the required format

current_date = now.strftime("%d-%m-%Y")
current_date

'11-08-2021'

In [10]:
# To get the hour

now.hour

12

In [11]:
# Increase the day to the next day
next_date = now + datetime.timedelta(days=1)
next_date

datetime.datetime(2021, 8, 12, 12, 2, 52, 82588, tzinfo=<DstTzInfo 'Asia/Kolkata' IST+5:30:00 STD>)

## 2. Get the vaccine slots based on this district code

In [26]:
# Now we get the vaccine slots here

parameters = {
    'district_id' : "540",
    "date" : current_date,
}
response = requests.get("https://cdn-api.co-vin.in/api/v2/appointment/sessions/public/findByDistrict",                              headers=headers, params=parameters)
print(response.status_code)

200


In [27]:
response.json()

{'sessions': [{'center_id': 711710,
   'name': 'Doddappanaickanoor BPHC WP CVC',
   'address': 'Doddappanaickanur Madurai Dt',
   'state_name': 'Tamil Nadu',
   'district_name': 'Madurai',
   'block_name': 'Usilampatti',
   'pincode': 625532,
   'from': '09:00:00',
   'to': '18:00:00',
   'lat': 9,
   'long': 77,
   'fee_type': 'Free',
   'session_id': 'ff2e8d32-d46f-4dd0-8769-3f1c3d57ca31',
   'date': '11-08-2021',
   'available_capacity': 0,
   'available_capacity_dose1': 0,
   'available_capacity_dose2': 0,
   'fee': '0',
   'allow_all_age': True,
   'min_age_limit': 18,
   'vaccine': 'COVISHIELD',
   'slots': ['09:00AM-11:00AM',
    '11:00AM-01:00PM',
    '01:00PM-03:00PM',
    '03:00PM-06:00PM']},
  {'center_id': 710674,
   'name': 'Valayankulam BPHC WP CVC',
   'address': 'Valayankulam Thotthukudi Main RdMadurai',
   'state_name': 'Tamil Nadu',
   'district_name': 'Madurai',
   'block_name': 'Thirupparankundram',
   'pincode': 625022,
   'from': '09:00:00',
   'to': '18:00:00',
 

## 3. Parse through this data

In [32]:
# Save this json response
data = response.json()

candidate_centers = []
# Parse through this json data and save the ones which have availability
for center in data['sessions'] :
    if (
        (center['vaccine'] in ['COVISHIELD', 'SPUTNIK V']) and
        (center['min_age_limit'] >= 18 ) and 
        (center['available_capacity_dose2'] > 0 )
    ) :
        cur_candidate = {
            'name' : center['name'],
            'num_dose_1' : center['available_capacity_dose1'],
            'fee' : center['fee'],
            'vaccine' : center['vaccine'],
            'min_age' : center['min_age_limit'],
            'slots' : center['slots']
        }
        candidate_centers.append(cur_candidate)

In [33]:
candidate_centers

[{'name': 'RIO CHILDREN HOSPITAL PVT LTD',
  'num_dose_1': 62,
  'fee': '780',
  'vaccine': 'COVISHIELD',
  'min_age': 18,
  'slots': ['11:00AM-12:00PM',
   '12:00PM-01:00PM',
   '01:00PM-02:00PM',
   '02:00PM-04:00PM']},
 {'name': 'Apollo Speciality Hospital',
  'num_dose_1': 402,
  'fee': '780',
  'vaccine': 'COVISHIELD',
  'min_age': 18,
  'slots': ['09:00AM-10:00AM',
   '10:00AM-11:00AM',
   '11:00AM-12:00PM',
   '12:00PM-03:00PM']},
 {'name': 'MMHRC Session Site',
  'num_dose_1': 3,
  'fee': '780',
  'vaccine': 'COVISHIELD',
  'min_age': 18,
  'slots': ['09:30AM-10:30AM',
   '10:30AM-11:30AM',
   '11:30AM-12:30PM',
   '12:30PM-05:00PM']}]

## 4. Generate the text version of these centers

In [ ]:
def generateText(candidate_centers) :
    """
    Given a list of candidate centers, converts these dictionary content to a text format
    """

    complete_text = ""

    for i, center in enumerate(candidate_centers) :
        complete_text += f"{i+1}<b><u>{center['name']} :</u></b>"
        complete_text += f"Doses Available : {center['num_dose_1']}"
        complete_text += f"Fee : {center['fee']}"
        complete_text += f"Vaccine : {center['vaccine']}"
        complete_text += f"Age group : {center['min_age_limit']}+ years"
        complete_text += f"Slots : {center['slots'][0]}"
        for j in range(1, len(center['slots'])) :
            complete_text += f"{center['slots'][j]}"
        complete_text += "---------------------"

# Logging activity

In [36]:
# Trim the log file such that it contains only the last 5 records
with open('./logs/status.log', 'r') as log_file:
    past_records = log_file.readlines()

past_records = past_records[-5:]

with open('./logs/status.log', 'w') as log_file:
    log_file.writelines(past_records)

# Twitter API


In [1]:
import tweepy

In [2]:
# Authenticate access

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(key, secret)

In [4]:
# Create API handler

api = tweepy.API(auth)

In [5]:
# Get the username of the reciever

user = api.get_user("user_name of recipient")
recipient_id = user.id_str
# This is to be stored as a repository secret

In [6]:
# Send a DM

api.send_direct_message(recipient_id, "Hello")

DirectMessage(_api=<tweepy.api.API object at 0x7ff65e895c40>, _json={'type': 'message_create', 'id': '1425342648353640452', 'created_timestamp': '1628663143610', 'message_create': {'target': {'recipient_id': '202973983'}, 'sender_id': '202973983', 'message_data': {'text': 'Hello', 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [], 'urls': []}}}}, type='message_create', id='1425342648353640452', created_timestamp='1628663143610', message_create={'target': {'recipient_id': '202973983'}, 'sender_id': '202973983', 'message_data': {'text': 'Hello', 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [], 'urls': []}}})